In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

print("All imports successful!")


In [184]:
GDP = pd.read_excel("Data/GDPperCapita.xlsx", engine='openpyxl')
Gini = pd.read_excel("Data/Gini Index.xlsx", engine='openpyxl')
Poverty = pd.read_excel("Data/Poverty per headcount.xlsx",engine='openpyxl')
Unemployment = pd.read_excel("Data/Unemployment levels.xlsx",engine='openpyxl')

In [ ]:
print(GDP.head())
print(GDP.info())
print(GDP.describe())
print(GDP.isnull().sum())


In [ ]:
#Create column 2024
GDP['2024'] = GDP['2023']
# Fill missing values
GDP[['2005','2006','2007']] = GDP[['2005','2006','2007']].fillna(GDP[['2005','2006','2007']].mean())
GDP['2008'] = GDP['2008'].fillna(GDP['2008'].mean())
GDP['2009'] = GDP['2009'].fillna(GDP['2009'].mean())
GDP['2010'] = GDP['2010'].fillna(GDP['2010'].mean())
#Interpolate first
GDP['2024'] = GDP.groupby('GeoAreaName')['2024'].transform(lambda x: x.fillna(x.mean()))
# Then fill remaining gaps with global mean
GDP[['2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021','2022', '2023']] = (
    GDP[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']]
    .fillna(GDP[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023','2024']].mean())
)
print(GDP.isnull().sum())


In [ ]:
GDP[['2005','2006','2007']] = GDP[['2005','2006','2007']].fillna(GDP[['2005','2006','2007']].mean())
GDP['2008'] = GDP['2008'].fillna(GDP['2008'].mean())
GDP['2009'] = GDP['2009'].fillna(GDP['2009'].mean())
GDP['2010'] = GDP['2010'].fillna(GDP['2010'].mean())
GDP[['2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021','2022', '2023']] = (
    GDP[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']]
    .fillna(GDP[['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023','2024']].mean())
)
print(GDP.isnull().sum())

In [ ]:
print(Gini.head())
print(Gini.info())
print(Gini.describe())
print(Gini.isnull().sum())

In [ ]:
# Select numeric year columns
yr_cols = [col for col in Gini.columns if col.isdigit()]

# Convert to numeric (just in case)
Gini[yr_cols] = Gini[yr_cols].apply(pd.to_numeric, errors='coerce')

# 1️⃣ Interpolate between years for each country
Gini[yr_cols] = Gini.groupby('GeoAreaName')[yr_cols].apply(lambda g: g.interpolate(axis=1)).reset_index(level=0, drop=True)

# 2️⃣ Fill remaining gaps with country mean
Gini[yr_cols] = Gini.groupby('GeoAreaName')[yr_cols].transform(lambda x: x.fillna(x.mean()))

# 3️⃣ Still empty? Fill with global mean
for col in yr_cols:
    Gini[col] = Gini[col].fillna(Gini[col].mean())

# Check results
print((Gini.isnull().sum() / len(Gini)) * 100)


In [ ]:
print(Poverty.head())
print(Poverty.info())
print(Poverty.describe())
print(Poverty.isnull().sum())


In [ ]:
# Select numeric year columns
cols = [col for col in Poverty.columns if col.isdigit()]

# Convert to numeric (just in case)
Poverty[cols] = Poverty[cols].apply(pd.to_numeric, errors='coerce')

# Interpolate between years for each country
Poverty[cols] = Poverty.groupby('GeoAreaName')[cols].apply(lambda g: g.interpolate(axis=1)).reset_index(level=0, drop=True)

# Fill remaining gaps with country mean
Poverty[cols] = Poverty.groupby('GeoAreaName')[cols].transform(lambda x: x.fillna(x.mean()))

# Still empty? Fill with global mean
for col in cols:
    Poverty[col] = Poverty[col].fillna(Poverty[col].mean())

# Check results
print((Poverty.isnull().sum() / len(Poverty)) * 100)


In [ ]:
print(Unemployment.head())
print(Unemployment.info())
print(Unemployment.describe())
print(Unemployment.isnull().sum())


In [ ]:
#Choose numeric year columns
Column = [col for col in Unemployment.columns if col.isdigit()]

# Convert to numeric (just in case)
Unemployment[Column] = Unemployment[Column].apply(pd.to_numeric, errors = 'coerce')

# Interpolate between years for each country
Unemployment[Column] = Unemployment.groupby('GeoAreaName')[Column]\
    .apply(lambda g:g.interpolate(axis = 1))\
    .reset_index(level=0, drop = True) #removes the extra group index and aligns it with the original DataFrame.

# Fill remaining gaps with country mean
Unemployment[Column] = Unemployment.groupby('GeoAreaName')[Column].transform(lambda x: x.fillna(x.mean()))

# Still empty? Fill with global mean
for col in Column:
    Unemployment[col] = Unemployment[col].fillna(Unemployment[col].mean())

# Check results
print(Unemployment.isnull().sum()/len(Unemployment)*100)

In [194]:
# First, select only numeric year columns from each dataset
GDP_cols = [col for col in GDP.columns if col.isdigit()]
Gini_cols = [col for col in Gini.columns if col.isdigit()]
Poverty_cols = [col for col in Poverty.columns if col.isdigit()]
Unemp_cols = [col for col in Unemployment.columns if col.isdigit()]

# Keep the country column + numeric columns for merging
GDP_merge = GDP[['GeoAreaName'] + GDP_cols]
Gini_merge = Gini[['GeoAreaName'] + Gini_cols]
Poverty_merge = Poverty[['GeoAreaName'] + Poverty_cols]
Unemp_merge = Unemployment[['GeoAreaName'] + Unemp_cols]

# Merge datasets one by one on 'GeoAreaName'
merged_df = GDP_merge.merge(Gini_merge, on='GeoAreaName', how='inner')
merged_df = merged_df.merge(Poverty_merge, on='GeoAreaName', how='inner')
merged_df = merged_df.merge(Unemp_merge, on='GeoAreaName', how='inner', suffixes=('_left', '_right'))

countries = merged_df['GeoAreaName'].unique()

# Prepare feature matrix X by dropping the non-numerical column
X = merged_df.drop(columns=['GeoAreaName'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X)


In [195]:
#Unsupervised learning,clustering

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 3,random_state= 35)
clusters = kmeans.fit_predict(X_Scaled)
merged_df['Cluster'] = clusters




In [ ]:
#Visualization of clusters
#Reduces dimensionality for 2D plotting
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=35)
X_pca = pca.fit_transform(X_Scaled)

merged_df['PCA1'] = X_pca[:,0]
merged_df['PCA2'] = X_pca[:,1]

# Plotting with Seaborn
plt.figure(figsize=(12,8))
sns.scatterplot(
    data=merged_df,
    x='PCA1',
    y='PCA2',
    hue='Cluster', #Colours by cluster
    palette='Set2',
    s=100 # stes size of each scatter point
)

In [ ]:
#Know what each cluster means using a cluster labeling system
merged_df['Cluster'] = clusters

key_features = X.columns.tolist() # Use the column names from X, not the DataFrame itself

# Compute cluster-wise averages but only for these columns
cluster_means = merged_df.groupby('Cluster')[key_features].mean().reset_index()
print("Cluster averages:\n", cluster_means)

#Mapping for my columns
column_map = {
    'GDP': '2024_x',
    'Poverty': '2024_y',
    'Unemployment': '2024_right',
    'Gini': '2024_left'  
}

# Function to label clusters
def label_cluster(row):
    score = 0

    # Higher is worse for Poverty, Gini, Unemployment
    score += (row[column_map['Poverty']] > merged_df[column_map['Poverty']].mean())
    score += (row[column_map['Gini']] > merged_df[column_map['Gini']].mean())
    score += (row[column_map['Unemployment']] > merged_df[column_map['Unemployment']].mean())

    # Lower is worse for GDP
    score += (row[column_map['GDP']] < merged_df[column_map['GDP']].mean())

    # Assign label based on score (0–4)
    if score >= 3:
        return 'High Poverty / High Inequality'
    elif score == 2:
        return 'Medium Poverty / Medium Inequality'
    else:
        return 'Low Poverty / Low Inequality'

# Apply labeling to cluster means
cluster_means['Cluster Label'] = cluster_means.apply(label_cluster, axis=1)
print(cluster_means[['Cluster', 'Cluster Label']])



       


In [ ]:
#Supervised Learning

#Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#Mapping for my columns
column_map = {
    'GDP': '2024_x',
    'Poverty': '2024_y',
    'Unemployment': '2024_right',
    'Gini': '2024_left'  # example if it has no suffix
}
# Prepare feature matrix X and target vector Y
X = merged_df.drop(columns=['GeoAreaName', 'Cluster', 'PCA1', 'PCA2', '2024_left'])
y = merged_df['2024_left']

# Split into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

# Initialize and train Random Forest Regressor
model = RandomForestRegressor(random_state=35)
model.fit(X_train,y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


